## ruGPT3 Finetuning

Попробуем натренировать сеть ruGPT3 на архивах старой компьютерной сети FIDONet.

Для начала, скачаем датасет (подготовлен [Дмитрием Сошниковым](https://soshnikov.com)):

In [1]:
!curl -L https://storage.yandexcloud.net/mypub/data/train.txt.gz | gzip -d > train.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  426M  100  426M    0     0  19.7M      0  0:00:21  0:00:21 --:--:-- 31.2M


Посмотрим на то, как выглядит датасет:

In [2]:
!head train.txt

<s>
From: Andrey Chekalin <Andrey.Chekalin@p20.f57.n5022.z2.fidonet.org>
Date: Mon, 07 Jul 2003 23:13:14 +0400
Subject: test
Organization: -= Alien Station =-
Lines: 7

                          Приветствую Вас, All!

test


Для ускорения возьмём часть датасета. Для полноценного обучения лучше брать весь.

In [3]:
!head -1000000 train.txt > train_sub.txt

Установим последние версии библиотек:

In [4]:
!pip install transformers accelerate evaluate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 M

Посмотрим на версию библиотеки transformers:

In [7]:
!pip freeze | grep transformers

transformers==4.41.2


Скачаем скрипт для тренировки сети для соответствующей версии (ниже пример для версии 4.41.2):

In [5]:
!curl -O https://raw.githubusercontent.com/huggingface/transformers/ab0f050b42d903f34d6eb97f3f8c0c07f0517ad2/examples/pytorch/language-modeling/run_clm.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28234  100 28234    0     0  87876      0 --:--:-- --:--:-- --:--:-- 87683


Запускаем обучение:

In [6]:
!python run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file train_sub.txt \
    --per_device_train_batch_size 1 \
    --num_train_epochs 1 \
    --block_size 2048 \
    --dataset_config_name plain_text \
    --do_train \
    --output_dir=models

2024-07-05 19:56:44.157971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 19:56:44.158026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 19:56:44.275382: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 19:56:44.518356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-05 19:56:46.644060: W tensorflow/comp

Попробуем генерацию:

In [9]:
from transformers import pipeline, AutoModelForCausalLM,AutoTokenizer
import torch

model_name = 'models/checkpoint-2500'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = pipeline(model=model,tokenizer=tokenizer,task="text-generation",device="cuda")

In [10]:
result = pipe("<s>Area: ru.anekdot",do_sample=True,max_length=1500)[0]['generated_text'].replace('\\n','\n')
result

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'<s>Area: ru.anekdot.ru, 28/июл/2009, 7\nOrganization: U$A station\nLines: 10\nBytes: 1924\nX-Original-Bytes: 1914\n\n╔═[■]═════════════════════════[ Hello All! ]═══════────────--─--─-\n\nБеда. Отказавшись от всех денег, он отказался от всего,\nчто у него есть. До самого самого конца не смог понять, как он выглядит\nдо этого момента.\n\t\t\n\n╚═[ Andrew Usachov ]═════════════════[ 28 июл 2009, 01:41 ]══───-─-\n\n\n'